<a href="https://colab.research.google.com/github/yunjeong-chang/2021_MachineLearning/blob/main/6%EC%A3%BC%EC%B0%A8_Assignment2_%EC%88%98%EB%A9%B4%EC%8B%9C%EA%B0%84%EC%97%90%20%EB%94%B0%EB%A5%B8%20%EC%9A%B0%EC%9A%B8%EC%A6%9D%20%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **<로지스틱 회귀를 통한 분류 문제>**

 : **수면시간에 따른 우울증 예측**


 * **데이터셋 설명**

   - 다트머스대학교 학생들에게 수집한 [설문조사](https://studentlife.cs.dartmouth.edu/)를 토대로 만들어진 데이터셋
   - 응답 결과의 자세한 내용은 [여기](https://studentlife.cs.dartmouth.edu/dataset.html)
   - 전처리 완료된 데이터

* **데이터파일 설명**

    - **train_X, test_X** : 수면 관련 Feature 14개 
        - sleep_time_mean
        - sleep_time_std
        - sleep_time_min
        - sleep_time_25
        - sleep_time_50
        - sleep_time_75
        - sleep_time_max
        - sleep_quality_mean
        - sleep_quality_std
        - sleep_quality_min
        - sleep_quality_25
        - sleep_quality_50
        - sleep_quality_75
        - sleep_quality_max

    - **train_Y** : 우울증 유/무를 표기
        - 0 : PHQ 9 종합 점수 5점이하
        - 1 : PHQ 9 종합 점수 6점이상
    
    - **sample_submit** : 제출 양식 (제출된 평가 데이터의 예측 분류 값의 성능은 Categorical Accuracy로 측정)

In [250]:
import pandas as pd
import numpy as np

In [251]:
#학습 데이터 불러오기
train_x = pd.read_csv('/content/train_X.csv')
train_y = pd.read_csv('/content/train_Y.csv')

#테스트 데이터 불러오기
test_x = pd.read_csv('/content/test_X.csv')

# 학습데이터 확인

      : 총 14개의 컬럼으로 이루어진 23개의 실수형 데이터, 라벨도 23개이고 0과 1로 이루어진 정수형 데이터, 결측치 없음

In [252]:
train_x

,sleep_time_mean,sleep_time_std,sleep_time_min,sleep_time_25,sleep_time_50,sleep_time_75,sleep_time_max,sleep_quality_mean,sleep_quality_std,sleep_quality_min,sleep_quality_25,sleep_quality_50,sleep_quality_75,sleep_quality_max
0,6.250000,0.689202,5.0,6.125,6.50,6.50,7.0,3.500000,0.836660,2.0,3.25,4.0,4.0,4.0
1,6.418182,1.169693,3.0,6.000,6.50,6.75,9.5,3.345455,0.672700,1.0,3.00,3.0,4.0,4.0
2,7.034483,1.127262,5.5,6.500,6.50,8.00,11.0,3.862069,0.347839,3.0,4.00,4.0,4.0,4.0
3,6.453846,1.070642,3.0,6.000,6.50,7.00,9.0,3.061538,0.768052,1.0,3.00,3.0,4.0,4.0
4,5.902439,1.817345,3.5,4.500,5.50,7.00,10.0,2.975610,0.961452,1.0,2.00,3.0,4.0,4.0
5,6.179487,1.205727,3.0,5.500,6.00,6.50,10.0,3.097561,0.916648,1.0,2.00,3.0,4.0,4.0
6,5.630435,0.741886,4.0,5.500,5.50,6.00,7.0,2.652174,0.884652,1.0,2.00,3.0,3.0,4.0
7,6.111111,0.609750,5.0,6.000,6.00,6.50,7.5,3.407407,0.572394,2.0,3.00,3.0,4.0,4.0
8,6.102564,1.539735,3.0,5.500,6.50,6.75,10.5,2.707317,1.078052,1.0,2.00,3.0,3.0,4.0
9,5.983333,0.636306,4.5,5.500,6.00,6.50,7.0,3.333333,0.660895,2.0,3.00,3.0,4.0,4.0


In [253]:
train_y

,label
0,0
1,0
2,0
3,1
4,0
5,0
6,1
7,0
8,1
9,0


In [254]:
print(train_x.shape)
print(train_y.shape)

(23, 14)
(23, 1)


In [255]:
train_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sleep_time_mean     23 non-null     float64
 1   sleep_time_std      23 non-null     float64
 2   sleep_time_min      23 non-null     float64
 3   sleep_time_25       23 non-null     float64
 4   sleep_time_50       23 non-null     float64
 5   sleep_time_75       23 non-null     float64
 6   sleep_time_max      23 non-null     float64
 7   sleep_quality_mean  23 non-null     float64
 8   sleep_quality_std   23 non-null     float64
 9   sleep_quality_min   23 non-null     float64
 10  sleep_quality_25    23 non-null     float64
 11  sleep_quality_50    23 non-null     float64
 12  sleep_quality_75    23 non-null     float64
 13  sleep_quality_max   23 non-null     float64
dtypes: float64(14)
memory usage: 2.6 KB


In [256]:
train_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   23 non-null     int64
dtypes: int64(1)
memory usage: 312.0 bytes


# 데이터 전처리

      : 우선은 필요 없음

In [257]:
#표준화 (정규화를 해도 성능은 비슷)
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scale.fit(train_x)
train_x_std = scale.transform(train_x)
test_x_std = scale.transform(test_x)

# 로지스틱 회귀 모델

In [258]:
from sklearn.linear_model import LogisticRegression

Logit = LogisticRegression(C=10, random_state=11, penalty='l1', solver='saga') #디폴트 파라미터 보다, solver=saga 쓰고 penalty=l1으로 바꿔주고 C=10으로 바꾸니까 성능 더 좋아짐
Logit.fit(train_x_std, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=11, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

# 예측

In [259]:
test_y_pred = Logit.predict(test_x_std)
test_y_pred

array([0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0])

# 예측값 저장

In [260]:
#제출 할 파일 불러오기
submit = pd.read_csv('/content/sample_submit.csv')
submit.head()

,id,label
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [261]:
#예측 값 파일에 저장
for i in range(len(test_y_pred)) :
  submit['label'][i] = test_y_pred[i]

submit


,id,label
0,1,0
1,2,0
2,3,0
3,4,1
4,5,0
5,6,0
6,7,1
7,8,0
8,9,1
9,10,0


In [262]:
#제출 파일에 최종 저장
submit.to_csv('submit_우울증.csv', index=False)